In [1]:
#stupid jupyter path is in the venv directory, so this is in MTG-INVESTOR-S-GRAIL
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np 
import tensorflow as tf 
from tensorflow.keras.utils import Sequence
import os
import time
import pandas as pd
import Data.sadness as sadness
from sklearn.utils import shuffle

%matplotlib inline


In [2]:
#making file with text from all cards
dftemp1 = pd.read_csv('Data/LSTMData.csv', sep=',')
dftemp1 = dftemp1[pd.notnull(dftemp1['text'])]
with open('Data/LSTMData.txt', 'w', encoding='UTF-8') as fileboi:
    for tupleboi in dftemp1.itertuples():
        if str(tupleboi[1]).count('nan') == 0:
            fileboi.write(str(tupleboi[1])+ '\n')


In [3]:
path2jank = 'Data/LSTMData.txt'
janktext = open(path2jank, 'rb').read().decode(encoding='utf-8')
vocablist = sorted(set(janktext).union(set(sadness.safe_string)))
num_chars = len(vocablist)
char2idx = {u:i+1 for i, u in enumerate(vocablist)}
idx2char = {value:key for key, value in char2idx.items()}

In [4]:
print ('{} unique characters'.format(num_chars))
dictyboi = {}
for char in char2idx:
    dictyboi[char] = 0
print(dictyboi)

99 unique characters
{'\n': 0, '\r': 0, ' ': 0, '!': 0, '"': 0, '&': 0, "'": 0, '(': 0, ')': 0, '+': 0, ',': 0, '-': 0, '.': 0, '/': 0, '0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, ':': 0, ';': 0, '?': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0, 'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0, 'Z': 0, '[': 0, ']': 0, '_': 0, 'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0, 'i': 0, 'j': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'o': 0, 'p': 0, 'q': 0, 'r': 0, 's': 0, 't': 0, 'u': 0, 'v': 0, 'w': 0, 'x': 0, 'y': 0, 'z': 0, '{': 0, '}': 0, '®': 0, '½': 0, 'á': 0, 'â': 0, 'é': 0, 'í': 0, 'ö': 0, 'ú': 0, 'û': 0, 'π': 0, '—': 0, '•': 0, '−': 0, '∞': 0, '☐': 0}


In [5]:
def convert_char(string):
    return [char2idx[char] for char in str(string)]

In [6]:
mask_value = 0
max_time_steps = shuffle(dftemp1).text.map(len).max()
x_train = np.full(shape=(len(dftemp1["text"]), max_time_steps, 1), fill_value=mask_value)
y_train = x_train
#y_train = np.zeros((len(dftemp1["oracle_text"]), max_time_steps, 1))
for s, x in enumerate(dftemp1.text.map(convert_char).to_numpy()):
    seq_len = len(x)
    x_train[s, 0:seq_len, :] = np.reshape(x, (seq_len, 1))
    y_train[s, 0:seq_len, :] = np.reshape(x, (seq_len, 1))

#x_train = np.reshape(x_train, (x_train.shape[0], max_time_steps))
print(x_train.shape, y_train.shape)

(47396, 769, 1) (47396, 769, 1)


In [7]:
set_length = x_train.shape[0]
cv_idx = int(set_length * .6)
test_idx = int(set_length * .8)


x_cv = x_train[cv_idx:test_idx, :]
y_cv = y_train[cv_idx:test_idx]

x_test = x_train[test_idx:, :]
y_test = y_train[test_idx:]

x_train = x_train[:cv_idx, :]
y_train = y_train[:cv_idx]

print("Training: {}, Cross val: {}, Testing: {}".format(x_train.shape, x_cv.shape, x_test.shape))

Training: (28437, 769, 1), Cross val: (9479, 769, 1), Testing: (9480, 769, 1)


In [8]:
def masking_layer(x):
    
    return tf.where(x >= 0, x, 0)

In [9]:
embedding_dim = 256

visible = tf.keras.layers.Input(shape=(max_time_steps, 1))
encoder = tf.keras.layers.Reshape((max_time_steps,))(visible)
encoder = tf.keras.layers.Embedding(num_chars+1, embedding_dim, mask_zero=True)(encoder)
#encoder = tf.keras.layers.Masking(mask_value=mask_value)(encoder)
encoder = tf.keras.layers.LSTM(1000, return_sequences = True)(encoder)
encoder = tf.keras.layers.LSTM(50)(encoder)
decoder = tf.keras.layers.RepeatVector(max_time_steps)(encoder)
decoder = tf.keras.layers.LSTM(500, return_sequences = True)(decoder)
decoder = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(decoder)
#decoder = tf.keras.layers.Lambda(masking_layer)(decoder)

model = tf.keras.models.Model(inputs=visible, outputs=decoder)
loss = tf.keras.losses.sparse_categorical_crossentropy
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 769, 1)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 769)               0         
_________________________________________________________________
embedding (Embedding)        (None, 769, 256)          25600     
_________________________________________________________________
lstm (LSTM)                  (None, 769, 1000)         5028000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                210200    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 769, 50)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 769, 500)          110200

In [10]:
for i in range (1, 2):   
    model.fit(x_train, y_train, epochs=5, batch_size=50)
    filepath = "Data/LSTMCheckpoints/" + "epoch{}.h5".format(str(i*50))
    model.save(filepath)

Train on 28437 samples
Epoch 1/50
28437/28437 [==============================] - 308s 11ms/sample - loss: 209.2018
Epoch 2/50
28437/28437 [==============================] - 304s 11ms/sample - loss: 138.4195
Epoch 3/50
13450/28437 [=============>................] - ETA: 2:41 - loss: 139.2501

KeyboardInterrupt: 

In [ ]:
model.save("LSTM_AEnc.h5")

In [ ]:
print("".join([idx2char[int(char[0])] if char[0] > 0 else "" for char in x_train[0:1][0].tolist()]))
print([idx2char[int(char[0])] if int(char[0]) > 0 else "" for char in (model.predict(x_train[0:1].astype(np.float32)).tolist()[0])])
print("".join([idx2char[int(char[0])] if int(char[0]) > 0 else "" for char in (model.predict(x_train[0:1].astype(np.float32)).tolist()[0])]))